<h1>Federated Learning with Grid Nodes </h1>
This notebook was made based on <a href="https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2010%20-%20Federated%20Learning%20with%20Secure%20Aggregation.ipynb">Part 10: Federated Learning with Encrypted Gradient Aggregation</a> tutorial

<h2>Import dependencies</h2>

In [2]:
import pickle
import torch
import torch.nn as nn
import grid as gr
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time

<h2>Setup config</h2>
Define Config parameters, init hook, etc...

In [3]:
hook = sy.TorchHook(torch)
class Parser:
    """Parameters for training"""
    def __init__(self):
        self.epochs = 1
        self.lr = 0.001
        self.test_batch_size = 8
        self.batch_size = 8
        self.log_interval = 10
        self.seed = 1
    
args = Parser()

torch.manual_seed(args.seed)
kwargs = {}

<h2>Load dataset</h2>

In [5]:
with open('../dataset/boston_housing.pickle','rb') as f:
    ((X, y), (X_test, y_test)) = pickle.load(f)

X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

<h2>Preprocessing</h2>

In [6]:
# preprocessing
mean = X.mean(0, keepdim=True)
dev = X.std(0, keepdim=True)
mean[:, 3] = 0. # the feature at column 3 is binary,
dev[:, 3] = 1.  # so I'd rather not standardize it
X = (X - mean) / dev
X_test = (X_test - mean) / dev
train = TensorDataset(X, y)
test = TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(test, batch_size=args.test_batch_size, shuffle=True, **kwargs)

<h2>Define Model</h2>

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc3 = nn.Linear(24, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
optimizer = optim.SGD(model.parameters(), lr=args.lr)

<h2>Connect with remote workers</h2>
<strong><me>Before this step, it is necessary to initialize the workers separately<me></strong>

In [8]:
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
alice =  gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
james = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
bob.connect()
alice.connect()
james.connect()

<h2>Ensure workers can communicate with each other</h2>
For this to be possible, you need to introduce them to each other.

In [9]:
bob.connect_grid_node(alice.uri, alice.id)
time.sleep(0.5)
bob.connect_grid_node(james.uri, james.id)
time.sleep(0.5)

alice.connect_grid_node(james.uri, james.id)
time.sleep(0.5)
alice.connect_grid_node(bob.uri, bob.id)
time.sleep(0.5)

james.connect_grid_node(alice.uri, alice.id)
time.sleep(0.5)
james.connect_grid_node(bob.uri, bob.id)

<h2>Populate workers with dataset and define training/test routines</h2>

In [10]:
compute_nodes = [bob, alice]


train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    train_distributed_dataset.append((data, target))

def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_distributed_dataset):
        worker = data.location
        model.send(worker)

        optimizer.zero_grad()
        # update the model
        pred = model(data)
        loss = F.mse_loss(pred.view(-1), target)
        loss.backward()
        optimizer.step()
        model.get()

        if batch_idx % args.log_interval == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))



def test():
    model.eval()
    test_loss = 0
    for data, target in test_loader:
        #data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.mse_loss(output.view(-1), target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))

<h2>Perform Train/test routines and measure performance</h2>

In [11]:
t = time.time()

for epoch in range(1, args.epochs + 1):
    train(epoch)

total_time = time.time() - t
print('Total', round(total_time, 2), 's')
test()

Train Epoch: 1 [0/51 (0%)]	Loss: 499.737000
Train Epoch: 1 [80/51 (20%)]	Loss: 444.433746
Train Epoch: 1 [160/51 (39%)]	Loss: 332.781433
Train Epoch: 1 [240/51 (59%)]	Loss: 133.071625
Train Epoch: 1 [320/51 (78%)]	Loss: 202.246490
Train Epoch: 1 [200/51 (98%)]	Loss: 16.212305
Total 138.32 s

Test set: Average loss: 25.9796

